In [1]:
import requests           # To request URLs
from time import sleep	  # To delay requesting process (avoid spam)
import datetime           # Deal with tricky dates formats
import json               # Because the API returns json data
import csv  

In [2]:
def request_properly(url):
	# Binary variable
	Success = False 
	# Systematic query
	while Success is False : 
		try :
    # Create an object containing our data
			request = requests.get(url)
    # Standard response codes given by web site servers on the 
    # internet. The codes help identify the cause of the problem when     
    # a web page or other resource does not load properly. 
			status_code = request.status_code
    # 200: successful query
			if status_code == 200:
				raw_data = request.text
				Success = True
    # 4XX: Client error
			elif status_code == 400:
				print(status_code, '\nRefresh your access TOKEN\n')
				Success = True
    # Wait 5secs and retry
		except Exception as e:
			print (e, '\n\t Please wait, Retrying...', datetime.datetime.now())
			sleep(5)
	# Return data, we clean later		
	return raw_data

In [3]:
def unicode_decode(text):
    try:
        return text.encode('utf-8').decode()
    except UnicodeDecodeError:
        return text.encode('utf-8')

In [4]:
# Parameters: check documentation
page = 'nytimes'
base_url = 'https://graph.facebook.com/v3.0/'
parameters = 'me?fields=id,name,conversations{messages.limit(10){created_time,message,from,id},id,link}'
access_token = 'EAAKHIpZBSTYwBABsoF3uzBSHOW4LKSJnj5eIr6p36smKVnB57wRNPCR2XseDTIW8U5PEnchJ0tyFXYMEZCU4iH3f9nZCiLu5EMi6xHZCzJG6gpt0qljWnOV924LmLXjtfr2euyB42BdWeZCYEh7pTN7Cbn45oJSm8dtZACI6ODyZAgZCokHdf5vJw2pg5GVaJeT1vVuNMZAtHHwZDZD'
# Concatenate parameters to build first URL
first_url = base_url + page + parameters + '&access_token=' + access_token

In [5]:
print first_url

https://graph.facebook.com/v3.0/nytimesme?fields=id,name,conversations{messages.limit(10){created_time,message,from,id},id,link}&access_token=EAAKHIpZBSTYwBABsoF3uzBSHOW4LKSJnj5eIr6p36smKVnB57wRNPCR2XseDTIW8U5PEnchJ0tyFXYMEZCU4iH3f9nZCiLu5EMi6xHZCzJG6gpt0qljWnOV924LmLXjtfr2euyB42BdWeZCYEh7pTN7Cbn45oJSm8dtZACI6ODyZAgZCokHdf5vJw2pg5GVaJeT1vVuNMZAtHHwZDZD


In [6]:
pagination = True
iter_num = 0
with open('facebook_posts_scraper.csv', 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(['id','name', 'conversation', 'message'])
    while pagination is True : 
        datas = json.loads(request_properly(first_url))
        for data in datas['data']:
            try :
                id_ = data['id']
                created_time = datetime.datetime.strptime(data['created_time'], '%Y-%m-%dT%H:%M:%S+0000')
                conversation = data['conversation']
                message = unicode_decode(data['message'])
            except KeyError:
                id, name, conversation, message = '','','',''


(ConnectionError(MaxRetryError("HTTPSConnectionPool(host='graph.facebook.com', port=443): Max retries exceeded with url: /v3.0/nytimesme?fields=id,name,conversations%7Bmessages.limit(10)%7Bcreated_time,message,from,id%7D,id,link%7D&access_token=EAAKHIpZBSTYwBABsoF3uzBSHOW4LKSJnj5eIr6p36smKVnB57wRNPCR2XseDTIW8U5PEnchJ0tyFXYMEZCU4iH3f9nZCiLu5EMi6xHZCzJG6gpt0qljWnOV924LmLXjtfr2euyB42BdWeZCYEh7pTN7Cbn45oJSm8dtZACI6ODyZAgZCokHdf5vJw2pg5GVaJeT1vVuNMZAtHHwZDZD (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10a2326d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))",),), '\n\t Please wait, Retrying...', datetime.datetime(2018, 12, 6, 19, 16, 10, 978249))
(400, '\nRefresh your access TOKEN\n')


UnboundLocalError: local variable 'raw_data' referenced before assignment

In [ ]:
Store data into csv: 4 cols
            csv_writer.writerow([id_, created_time, conversation, message])
            iter_num+=1
            first_url = datas['paging']['next']

In [5]:
# Request data, paginate and write into csv file

pagination = True
iter_num = 0
with open('facebook_posts_scraper.csv', 'w') as f:

    # 1. Create an object for writing data into csv 
    csv_writer = csv.writer(f)
    csv_writer.writerow(['id','created_time', 'shares', 'description'])
    # 2. Loop over pages
    while pagination is True : 
        datas = json.loads(request_properly(first_url))
        for data in datas['data']:
            try :
                id_ = data['id']
                created_time = datetime.datetime.strptime(data['created_time'], '%Y-%m-%dT%H:%M:%S+0000')
                shares = data['shares']['count']
                description = unicode_decode(data['description'])
            except KeyError:
                description, id_, created_time, shares = '','','',''

    # 3. Store data into csv: 4 cols
            csv_writer.writerow([id_, created_time, shares, description])
            iter_num+=1
            first_url = datas['paging']['next']
            # Number of posts scraped Indicator
            if (iter_num%100) == 0 :
                print('{} Posts scraped... \t datetime : {}'.format(iter_num, datetime.datetime.now()))
            # Finish pagination 
            if not (datas['paging']['next']) or (iter_num == 500):
                pagination = False 
        
    # 4. Stop program when reach number of post we want
print '\n {} Posts scraped SUCCESSFULY ! \t datetime : {}'.format(iter_num, datetime.datetime.now())       

(400, '\nRefresh your access TOKEN\n')


UnboundLocalError: local variable 'raw_data' referenced before assignment

In [ ]:

import pandas as pd  
path = '/Users/../?'
df = pd.read_csv(path)
print df
print df.shape
print df.isnull().sum()

In [ ]:
{
    "name" : "Admin",
    "age" : 36,
    "rights" : [ "admin", "editor", "contributor" ]
}